In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
path = '/content/drive/MyDrive/'
os.chdir(path)

In [85]:
import pandas as pd
import os

df = pd.read_csv("./Processed_Daytime/merge_train_test.csv")
correct_names = []
for image_name in os.listdir("./Processed_Daytime/images_rotate"):
  correct_names.append(image_name)
# print(correct_names)
# for line in df[['file_Id', "PM2.5"]]:
# print(df[['file_Id', "PM2.5"]])
with open("./Processed_Daytime/merge_train_test_final.csv", mode="w") as file:
  # 写入表头
  file.write('file_Id,PM2.5\n')

  for index in range(len(df)):
    temp_str = str(df.loc[index, ['file_Id']]).split(" ")[4]
    file_Id = temp_str.split('\n')[0]
    temp_PM = str(df.loc[index, ['PM2.5']]).split(" ")[4]
    PM = temp_PM.split('\n')[0]
    file.write(f'{file_Id},{PM}\n')
  # print(PM)
  # if file_Id in correct_names:
  #   df.loc[index].to_csv("./Processed_Daytime/merge_train_test_final.csv", mode='a+')
print("Finished")

Finished


In [86]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("./Processed_Daytime/merge_train_test_final.csv")
X_train, X_test, y_train, y_test = train_test_split(df['file_Id'], df['PM2.5'], test_size=0.2)
df_test = pd.DataFrame(columns=['file_Id', 'PM2.5'])
df_test['file_Id'] = X_test
df_test['PM2.5'] = y_test
# print(df_test)
df_train = pd.DataFrame(columns=['file_Id', 'PM2.5'])
df_train['file_Id'] = X_train
df_train['PM2.5'] = y_train

train_csv_dir = r"/content/drive/MyDrive/Processed_Daytime/anno/processed_train.csv"
test_csv_dir = r"/content/drive/MyDrive/Processed_Daytime/anno/processed_test.csv"

df_test.to_csv(test_csv_dir)
df_train.to_csv(train_csv_dir)
print("Finished!")

Finished!


In [64]:
import pandas as pd
import numpy as np
import torch
import math
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn.functional as F
import time
import os
from torchvision import models
import cv2
import PIL
from PIL import Image
import cv2
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.autograd import Variable
import matplotlib.pyplot as plt

def DarkChannel(im, win=15):
  """求暗通道图
  input:three channels rgb, window size of fusing image
  output:dark channel image
  """
  if isinstance(im, PIL.Image.Image):
    im = cv2.cvtColor(np.asarray(im), cv2.COLOR_RGB2BGR)
    
  b, g, r = cv2.split(im)
  dc = cv2.min(cv2.min(r, g), b)
  kernal = cv2.getStructuringElement(cv2.MORPH_RECT, (win, win))
  dark = cv2.erode(dc, kernal)
  return Image.fromarray(dark, mode='L')

def Saturation(im):
  im = Image.fromarray(im)
  img_hsv = im.convert('HSV')
  h, s, v = img_hsv.split()
  return s

class ImgDataset(Dataset):
  def calc_image_entropy(self, img):
    entropys = []

    hist = cv2.calcHist([img], [0], None, [256], [0, 255])
    total_pixels = img.shape[0] * img.shape[1]

    for frequency in hist:
      p = frequency / total_pixels
      if p == 0:
        pixel_entropy = 0
      else:
        pixel_entropy = -1 * p * (np.log(p) / np.log(2))
      entropys.append(pixel_entropy)
    
    return np.sum(entropys)

  # 调整图像的亮度，以区分效果好的图和不好的图
  def tunning_img_light(self, img, w):
    img_ = img.copy().astype(np.int16)

    bias = np.ones_like(img_)
    bias *= 255
    
    temp = np.add(w * np.subtract(img_, bias), bias)
    img_ = np.minimum(temp, bias)
    
    img_ = np.clip(img_, 0, 255)
    return img_.astype(np.uint8)

  def get_info(self, data_dir):
    data_info = list()
    judge = data_dir.split("/")[-1]
    if judge == "train":
      path = r"/content/drive/MyDrive/Processed_Daytime/anno/processed_train.csv"
    else:
      path = r"/content/drive/MyDrive/Processed_Daytime/anno/processed_test.csv"
    df = pd.read_csv(path)
    df_xy = df[["file_Id", "PM2.5"]]
    for index in range(len(df_xy)):
      temp_str = str(df_xy.loc[index, ["file_Id"]]).split(" ")[4]
      final_path = temp_str.split('\n')[0]
      img_path = os.path.join(data_dir, final_path)
      label = df_xy.loc[index, ["PM2.5"]]
      # normalization
      label = (label - 1) / (262 - 1)      
      data_info.append((img_path, label))
    return data_info

  def __init__(self, data_dir, transforms=None):
    self.data_info = self.get_info(data_dir)
    self.transforms = transforms
    
  def __getitem__(self, index):
    features = list()
    labels = list()
    w = list(range(1, 21))
    img_path, label = self.data_info[index]
    labels.append(float(label))
    # print(img_path)
    img_ = cv2.imread(img_path)
    dc = DarkChannel(img_)
    saturation = Saturation(img_)
    img_ = np.stack([dc, saturation])
    for element in w:
      dc_entropy = self.calc_image_entropy(self.tunning_img_light(img_[0], element))[0]
      saturation_entropy = self.calc_image_entropy(self.tunning_img_light(img_[0], element))[0]
      features.append(dc_entropy)
      features.append(saturation_entropy)
    # 转换数据类型
    # features = torch.from_numpy(np.array(features))
    features = Image.fromarray(np.array(features).astype(np.uint8))
    features = features.convert('RGB')
    w, h = features.size
    # print("图像宽度：", w)
    # print("图像高度：", h)
    features = self.transforms(features)
    w_, h_ = features.shape[-2:]
    # print("转换后图像宽度：", w_)
    # print("转换后图像高度：", h_)
    labels = torch.from_numpy(np.array(labels))
    # print(labels.shape)
    return features, labels    
  
  def __len__(self):
    return len(self.data_info)

# path = r"/content/drive/MyDrive/Daytime/merge_train_test.csv"
# df = pd.read_csv(path)
# X_train, X_test, y_train, y_test = train_test_split(df['file_Id'], df['PM2.5'], test_size=0.2)
# print(X_train)
# print("------------------")
# print(y_train)

transforms = transforms.Compose(
    [
      transforms.Resize((224, 224)),
      transforms.ToTensor()
    ]
)

train_dir = r"/content/drive/MyDrive/Processed_Daytime/train"
test_dir = r"/content/drive/MyDrive/Processed_Daytime/test"

train_data = ImgDataset(data_dir=train_dir,transforms=transforms)
test_data = ImgDataset(data_dir=test_dir,transforms=transforms)

# 做32次___getitem__()
train_data_loader = DataLoader(dataset=train_data, batch_size=32, shuffle=True)
test_data_loader = DataLoader(dataset=test_data, batch_size=32)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

resnet_101 = models.resnet101(pretrained=True)
resnet_101.fc = nn.Linear(2048, 1)
resnet_101 = resnet_101.to(device)

EPOCHS = 100
LR = 1e-4

loss_func = nn.MSELoss()
loss_func = loss_func.to(device)
optimizer = optim.Adam(resnet_101.parameters(), lr=LR)

# --------------record the loss and accuracy----------
total_train_loss = []

def train():
  for epoch in range(EPOCHS):
      train_loss = 0.0
      resnet_101.train()
      for images, labels in train_data_loader:
        images = images.to(device)
        labels = labels.to(device)
        # print(labels)
        if hasattr(torch.cuda, 'empty_cache'):
	        torch.cuda.empty_cache()
        
        outputs = resnet_101(images)
        # print(outputs.shape)
        outputs = outputs.type(torch.float64)
        # print(outputs)
        # print("-------")
        # print(labels)
        loss = loss_func(outputs, labels)
        
        # 反向传播前要把上一次的梯度清0
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
      print("Epoch {}, Training loss: {:.6f}".format(epoch, train_loss))
      total_train_loss.append(train_loss)

def test():
  with torch.no_grad():
    test_loss = 0.0
    resnet_101.eval()
    for images, labels in test_data_loader:
      images = images.to(device)
      labels = labels.to(device)
      
      outputs = resnet_101(images)
      loss = loss_func(outputs, images)
      test_loss += loss
    print("The test loss: {:.6f}".format(test_loss))  

if __name__ == "__main__":
  train()
  test()

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.9/dist-packages/numpy/core/fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, ou

ValueError: ignored